# 4.4 Statistical Modeling Tutorial

This notebook covers key concepts in statistical modeling including:
- Model Selection
- Polynomial Regression
- Logistic Regression
- Regularization
- Model Interpretation

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

# Set random seed for reproducibility
np.random.seed(42)

## 1. Model Selection

Let's demonstrate how to select appropriate models based on data characteristics.

In [ ]:
# Generate non-linear data
X = np.linspace(-3, 3, 100).reshape(-1, 1)
y = 0.5 * X.ravel()**3 - X.ravel()**2 + 2 + np.random.normal(0, 1, 100)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compare different polynomial degrees
degrees = [1, 2, 3, 5]
plt.figure(figsize=(15, 5))

for i, degree in enumerate(degrees, 1):
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree)
    X_poly_train = poly.fit_transform(X_train)
    X_poly_test = poly.transform(X_test)
    
    # Fit model
    model = LinearRegression()
    model.fit(X_poly_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_poly_test)
    mse = mean_squared_error(y_test, y_pred)
    
    # Plot
    plt.subplot(1, 4, i)
    plt.scatter(X_test, y_test, color='blue', alpha=0.5, label='Actual')
    
    # Sort for smooth curve plotting
    X_plot = np.sort(X_test, axis=0)
    X_plot_poly = poly.transform(X_plot)
    y_plot = model.predict(X_plot_poly)
    
    plt.plot(X_plot, y_plot, color='red', label=f'Degree {degree}')
    plt.title(f'Polynomial (d={degree})\nMSE: {mse:.2f}')
    plt.legend()

plt.tight_layout()
plt.show()

## 2. Polynomial Regression

Let's explore polynomial regression in more detail.

In [ ]:
# Create polynomial regression pipeline
def create_poly_pipeline(degree):
    return Pipeline([
        ('poly', PolynomialFeatures(degree=degree)),
        ('linear', LinearRegression())
    ])

# Fit models and evaluate using cross-validation
degrees = range(1, 8)
cv_scores = []

for degree in degrees:
    pipeline = create_poly_pipeline(degree)
    scores = -cross_val_score(pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
    cv_scores.append(scores.mean())

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(degrees, cv_scores, marker='o')
plt.xlabel('Polynomial Degree')
plt.ylabel('Mean Squared Error (CV)')
plt.title('Model Complexity vs. Error')
plt.grid(True)
plt.show()

# Print best degree
best_degree = degrees[np.argmin(cv_scores)]
print(f"Best polynomial degree: {best_degree}")

## 3. Logistic Regression

Let's implement logistic regression for classification.

In [ ]:
# Generate classification data
n_samples = 200
X_class = np.random.randn(n_samples, 2)
y_class = (X_class[:, 0] + X_class[:, 1] > 0).astype(int)

# Split data
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42
)

# Fit logistic regression
log_reg = LogisticRegression()
log_reg.fit(X_train_class, y_train_class)

# Make predictions
y_pred_class = log_reg.predict(X_test_class)
accuracy = accuracy_score(y_test_class, y_pred_class)

# Plot decision boundary
def plot_decision_boundary(X, y, model):
    h = 0.02  # step size in mesh
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.contourf(xx, yy, Z, alpha=0.4)
    plt.scatter(X[:, 0], X[:, 1], c=y, alpha=0.8)

plt.figure(figsize=(10, 6))
plot_decision_boundary(X_class, y_class, log_reg)
plt.title(f'Logistic Regression Decision Boundary\nAccuracy: {accuracy:.2f}')
plt.show()

# Print confusion matrix
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## 4. Regularization

Let's compare different regularization techniques.

In [ ]:
# Generate data with many features
n_samples, n_features = 100, 20
X_reg = np.random.randn(n_samples, n_features)
true_coefficients = np.zeros(n_features)
true_coefficients[:5] = [1, 0.5, -0.8, 0.3, -0.4]  # Only first 5 features are relevant
y_reg = np.dot(X_reg, true_coefficients) + np.random.normal(0, 0.1, n_samples)

# Split data
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Compare models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge (L2)': Ridge(alpha=1.0),
    'Lasso (L1)': Lasso(alpha=1.0)
}

# Fit models and collect coefficients
coefficients = {}
test_scores = {}

for name, model in models.items():
    model.fit(X_train_reg, y_train_reg)
    coefficients[name] = model.coef_
    test_scores[name] = model.score(X_test_reg, y_test_reg)

# Plot coefficients
plt.figure(figsize=(12, 6))
x = np.arange(n_features)

for name, coef in coefficients.items():
    plt.plot(x, coef, 'o-', label=f'{name} (R² = {test_scores[name]:.3f})')

plt.plot(x, true_coefficients, 'k--', label='True Coefficients')
plt.xlabel('Feature Index')
plt.ylabel('Coefficient Value')
plt.title('Comparison of Regularization Methods')
plt.legend()
plt.grid(True)
plt.show()

## 5. Model Interpretation

Let's explore techniques for interpreting our models.

In [ ]:
# Feature importance analysis
def plot_feature_importance(model, feature_names=None):
    if feature_names is None:
        feature_names = [f'Feature {i}' for i in range(len(model.coef_))]
    
    importance = np.abs(model.coef_)
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance
    })
    
    feature_importance = feature_importance.sort_values('Importance', ascending=True)
    
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(importance)), feature_importance['Importance'])
    plt.yticks(range(len(importance)), feature_importance['Feature'])
    plt.xlabel('Absolute Coefficient Value')
    plt.title('Feature Importance')
    plt.tight_layout()

# Plot feature importance for each model
for name, model in models.items():
    plt.figure(figsize=(10, 6))
    plot_feature_importance(model)
    plt.title(f'Feature Importance - {name}')
    plt.show()

## Practice Exercises

1. Experiment with different regularization strengths (alpha values) and observe their effects.

2. Implement cross-validation for hyperparameter tuning.

3. Create a more complex classification problem and solve it using logistic regression.

4. Compare polynomial regression with other non-linear modeling techniques.

5. Develop a comprehensive model evaluation framework including multiple metrics.